### Creating Dataloaders

In [1]:
import tensorflow as tf
import keras_tuner
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,SGD

train_ds = keras.utils.image_dataset_from_directory(
    directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\train',
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    color_mode="rgb",
    seed=123,
    image_size=(240, 240))

test_ds = keras.utils.image_dataset_from_directory(
    directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\test',
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    color_mode="rgb",
    seed=123,
    image_size=(240, 240))

Found 1512 files belonging to 2 classes.
Found 190 files belonging to 2 classes.


### Hyper parameter tuning

In [2]:
from tensorflow.keras.optimizers import Adam,SGD

def build_model(hp):
    resnet_model = Sequential()

    pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                       input_shape=(240,240,3),
                       pooling='avg',classes=2,
                       weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False

    resnet_model.add(pretrained_model)
    resnet_model.add(Flatten())
    resnet_model.add(Dense(1, activation='sigmoid'))
    optimizer=hp.Choice('optimizer', values=['sgd', 'adam'])                
    # Conditional for each optimizer
    if optimizer == 'adam':
        learning_rate = hp.Choice("learning_rate", values=[0.001, 0.01, 0.1])
        resnet_model.compile(optimizer=Adam(lr=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
    elif optimizer == 'sgd':
        learning_rate = hp.Choice("learning_rate", values=[0.001, 0.01, 0.1])
        resnet_model.compile(optimizer=SGD(learning_rate=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
    return resnet_model

build_model(keras_tuner.HyperParameters())

In [3]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    max_retries_per_trial=1,
    max_consecutive_failed_trials=3,
    overwrite=True,
    directory=r"D:\\",
    project_name="hyper_parameter_results_thesis")

In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'adam'], 'ordered': False}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.01, 0.1], 'ordered': True}


In [5]:
tuner.search(train_ds, epochs=5, validation_data=test_ds)

Trial 3 Complete [00h 09m 45s]
val_accuracy: 0.7368420958518982

Best val_accuracy So Far: 0.74210524559021
Total elapsed time: 00h 29m 51s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


### Results  of Hyper parameter tuning

In [6]:
tuner.results_summary()

Results summary
Results in D:\\hyper_parameter_results_thesis
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
optimizer: adam
learning_rate: 0.1
Score: 0.74210524559021

Trial 1 summary
Hyperparameters:
optimizer: adam
learning_rate: 0.001
Score: 0.74210524559021

Trial 2 summary
Hyperparameters:
optimizer: sgd
learning_rate: 0.1
Score: 0.7368420958518982
